In [ ]:
import pprint as pp
import numpy as np
import time
import os
import sys
import random


import matplotlib.pyplot as plt
%matplotlib inline 
import matplotlib
from matplotlib.patches import Patch

import shutil
from numpy import genfromtxt
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable

torch.set_default_dtype(torch.float64)

import itertools
import numpy.linalg  as lin

import cProfile, pstats

from collections import OrderedDict
torch.set_num_threads(1) #cpu num
from fractions import Fraction
import seaborn as sns
import pandas as pd
import math
from collections import defaultdict
import seaborn as sns
from matplotlib.lines import Line2D

In [ ]:
%cd ../AllLogs

In [ ]:
logfolder = '%s/reg/%s/'

In [ ]:
DNAME = ['ijcnn1','rcv1','real-sim','news20','covtype']
ENDEP = [20,30,20,40,20]
TICK=[5,10,5,10,5]
ALGO_NAME = ['AI-SARAH', 'SARAH', 'SARAH+', 'SVRG', 'Adam', 'SGD w/m']

# Legend for Figure 3

In [ ]:
palette = sns.color_palette("tab10")[:6]
hue_order = ['AI-SARAH','SARAH','SARAH+','SVRG','Adam','SGD w/m']
handles=[Line2D([0],[0],color=palette[0],linewidth=3,dashes=[2,2])]
handles+=[Line2D([0],[0],color=palette[i],linewidth=6) for i in range(1,len(hue_order))]
fig,ax=plt.subplots(1,1,figsize=(15,1))
ax.legend(handles=handles,labels=hue_order,fancybox=True,fontsize=10,loc='center',\
         ncol=len(hue_order),prop={'size': 15})
ax.axis('off')
fig.tight_layout()
plt.show()

# Figure 3

In [ ]:
df={}
for dd in range(len(DNAME)):
    dname = DNAME[dd]
    DF = []
    endep = ENDEP[dd]    
    for al in range(len(ALGO_NAME)):
        alg_name = ALGO_NAME[al]
        temp_logfolder = logfolder%(dname,algo)
        cnt=0
        for f in os.listdir(temp_logfolder):
            if '.tar' not in f or 'DONE' in f or 'RUN' in f:
                continue
            temp = torch.load(temp_logfolder+f)
            if '.tar' not in f or 'DONE' in f or 'RUN' in f:
                continue
            temp = torch.load(temp_logfolder+f)
            parm = temp['parm']
            if alg_name=='AI-SARAH' and parm[2]!=1/32:
                continue
            stat = temp['stat']
            stat = [si for si in stat if si[0]<=endep]
            stat = np.array(stat)
            hist = temp['hist']
            hist = [si for si in hist if si[0]<=endep]
            hist = np.array(hist)
            ep = list(hist[:,0])
            grad = list(hist[:,2])
            grad = [np.min(grad[:oi]) for oi in range(1,len(grad)+1)]
            DF+=[[alg_name,cnt,ei,gi] for ei,gi in zip(ep,grad)]
            cnt+=1
    df[dname]=pd.DataFrame(data=DF,columns=['algo','cnt','ep','grad'])  

In [ ]:
ENDEP = [20,30,20,40,20]
STEP = [10,15,10,20,10]

CRIT = [[5,ti,ei] for ti,ei in zip(STEP,ENDEP)]

sns.set(context="paper",font='serif')
sns.set_style("whitegrid")
subdf2 = subdf.loc[subdf.algo!='AI-SARAH']
palette = sns.color_palette("tab10")[:6]

aisarah_line=[Line2D([0],[0],color=palette[0],linewidth=3)]

fig, ax = plt.subplots(1,len(DNAME),figsize=(3*len(DNAME),3),sharey=False)
for d in range(len(DNAME)):
    crit = CRIT[d]
    subdf = df[DNAME[d]].copy(deep=True)
    subdf['ep_round']=np.round(subdf['ep'])
    cond = [(subdf.ep_round<=ci) for ci in crit]
    subdf['ep_step']=np.select(cond,crit)
    subdf = subdf.groupby(['algo','cnt','ep_step']).agg({'grad':'min'}).reset_index()
    subdf2=subdf.loc[subdf.algo!='AI-SARAH']
    
    aisarah = subdf.loc[subdf.algo=='AI-SARAH'].groupby(['algo','ep_step']).agg({'grad':'min'}).reset_index()
    
    hue_order = ['SARAH','SARAH+','SVRG','Adam','SGD w/m']
    sns.boxenplot(data=subdf2, x="ep_step", y="grad", hue="algo", hue_order=hue_order,\
                      palette=palette[1:], \
                      linewidth=2,ax=ax[d])
    for ci in range(len(crit)):
        ax[d].axhline(y=aisarah.loc[aisarah.ep_step==crit[ci]]['grad'].values.item(),\
                      xmin=ci*0.33, xmax=(ci+1)*0.33,color=palette[0],ls='--',lw=2)
    ax[d].set_yscale('log')
    ax[d].set_ylabel('')
    ax[d].set_xlabel('Effective Pass',fontsize=12)
    ax[d].set_title(DNAME[d],fontsize=15)
    if d>=0:
        ax[d].legend('',frameon=False)
ax[0].set_ylabel(r'$||\nabla P(w)||^2$',fontsize=12)
fig.tight_layout()
plt.show()

In [ ]:
sns.set(context="paper",font='serif')
sns.set_style("whitegrid")
palette = sns.color_palette("tab10")[:6]

aisarah_line=[Line2D([0],[0],color=palette[0],linewidth=3)]

fig, ax = plt.subplots(1,len(DNAME),figsize=(3*len(DNAME),3),sharey=False)
for d in range(len(DNAME)):
    subdf = df2[DNAME[d]].copy(deep=True)
    subdf['tp_round']=np.round(subdf['tp'])
    timelist=sorted(list(subdf.tp_round.unique()))
    crit=[10.0,timelist[int(len(timelist)/2)-1],timelist[-1]]
    SUBDF=[]
    for ci in crit:
        temp=subdf.loc[subdf.tp_round<=ci].groupby(['algo','cnt']).agg({'grad':'min'}).reset_index()
        temp['tp_step']=ci
        SUBDF.append(temp)
    SUBDF=pd.concat(SUBDF, ignore_index=True, sort=False)
    subdf2=SUBDF.loc[subdf.algo!='AI-SARAH']
    
    aisarah = SUBDF.loc[SUBDF.algo=='AI-SARAH'].groupby(['algo','tp_step']).agg({'grad':'min'}).reset_index()
    
    hue_order = ['SARAH','SARAH+','SVRG','Adam','SGD w/m']
    sns.boxenplot(data=subdf2, x="tp_step", y="grad", hue="algo", hue_order=hue_order,\
                      palette=palette[1:], \
                      linewidth=2,ax=ax[d])
    for ci in range(len(crit)):
        ax[d].axhline(y=aisarah.loc[aisarah.tp_step==crit[ci]]['grad'].values.item(),\
                      xmin=ci*0.33, xmax=(ci+1)*0.33,color=palette[0],ls='--',lw=2)
    ax[d].set_yscale('log')
    ax[d].set_ylabel('')
    ax[d].set_xlabel('Wall Clock (sec)',fontsize=12)
    if d>=0:
        ax[d].legend('',frameon=False)
ax[0].set_ylabel(r'$||\nabla P(w)||^2$',fontsize=12)

fig.tight_layout()
plt.show()

In [ ]:
exit(0)